# Filigreen accounting export — data views

Views of `raw/filigreen_accounting_export.csv` (Shore of Filigreen LLC).

Regenerate the CSV with: `python3 scripts/export_accounting_csv.py`

In [ ]:
import pandas as pd
from pathlib import Path

repo = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
csv_path = repo / 'raw' / 'filigreen_accounting_export.csv'
df = pd.read_csv(csv_path)
df['Section'] = df['Section'].str.strip()
sections = df['Section'].unique()
print(f"Loaded {len(df)} rows. Sections: {list(sections)}")

## 1. Member allocations

In [ ]:
members = df[df['Section'] == 'Member Allocations'].copy()
cols = ['Member Name', 'Member Type', 'Total Capital', 'Loan 1 Allocation %', 'Loan 2 Allocation %']
members = members[[c for c in cols if c in members.columns]].dropna(how='all', axis=1)
members['Total Capital'] = pd.to_numeric(members['Total Capital'], errors='coerce')
members

In [ ]:
ax = members.set_index('Member Name')['Total Capital'].plot(kind='bar', figsize=(8, 4), title='Total capital by member')
ax.set_ylabel('Total Capital ($)')
ax.figure.tight_layout()

## 2. Loan performance (monthly)

In [ ]:
perf = df[df['Section'] == 'Loan Performance'].copy()
cols = ['Month', 'Loan 1 Interest', 'Loan 1 Principal', 'Loan 1 Balance', 'Loan 2 Interest', 'Loan 2 Principal', 'Loan 2 Balance']
perf = perf[[c for c in cols if c in perf.columns]]
for c in perf.columns:
    if c != 'Month':
        perf[c] = pd.to_numeric(perf[c], errors='coerce')
perf

In [ ]:
ax = perf.set_index('Month')[['Loan 1 Interest', 'Loan 2 Interest']].plot(kind='bar', figsize=(10, 4), title='Monthly interest by loan')
ax.set_ylabel('Interest')
ax.legend(title='Loan')
ax.figure.tight_layout()

## 3. Loan details

In [ ]:
details = df[df['Section'] == 'Loan Details'][['Loan', 'Detail Field', 'Detail Value']].dropna(subset=['Detail Field'])
details = details[details['Detail Field'].str.strip() != '']
details

## 4. Transactions (on-chain)

In [ ]:
tx = df[df['Section'] == 'Transactions'].copy()
tx['Amount'] = pd.to_numeric(tx['Amount'], errors='coerce')
tx['Date'] = pd.to_datetime(tx['Date'], errors='coerce')
cols = ['Date', 'Chain', 'Amount', 'Asset Symbol', 'From Address', 'To Address']
tx[cols].sort_values('Date', ascending=False)

In [ ]:
print('Transaction count by chain:')
print(tx['Chain'].value_counts().to_string())
print()
print('Sum of Amount by chain:')
print(tx.groupby('Chain')['Amount'].sum().to_string())

In [ ]:
print('Sum of Amount by asset:')
by_asset = tx.groupby('Asset Symbol')['Amount'].agg(['sum', 'count']).sort_values('sum', ascending=False)
by_asset.columns = ['Total Amount', 'Count']
by_asset

In [ ]:
monthly = tx.set_index('Date').resample('ME')['Amount'].sum()
ax = monthly.plot(figsize=(10, 4), title='Transaction volume by month', marker='o')
ax.set_ylabel('Amount')
ax.figure.tight_layout()